Nama : **Fauzan Ahsanudin Alfikri**

Kelas : **DS-47-03**

NIM : **103052300003**

In [29]:
import os
import base64
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import pandas as pd
from google.colab import files

print("✓ Libraries berhasil diimport!")


✓ Libraries berhasil diimport!


In [30]:
def upload_file():
    print("📤 Silakan pilih file untuk diupload...")
    uploaded = files.upload()

    if not uploaded:
        return None

    filename = list(uploaded.keys())[0]
    print(f"✓ File berhasil diupload: {filename}")
    return filename


In [31]:
def derive_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())


In [32]:
def encrypt_text(plaintext: str, password: str) -> dict:
    salt = os.urandom(16)
    iv = os.urandom(16)

    key = derive_key(password, salt)

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'ciphertext': base64.b64encode(ciphertext).decode('utf-8'),
        'iv': base64.b64encode(iv).decode('utf-8'),
        'salt': base64.b64encode(salt).decode('utf-8')
    }


In [33]:
def decrypt_text(encrypted_data: dict, password: str) -> str:
    ciphertext = base64.b64decode(encrypted_data['ciphertext'])
    iv = base64.b64decode(encrypted_data['iv'])
    salt = base64.b64decode(encrypted_data['salt'])

    key = derive_key(password, salt)

    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    return plaintext.decode('utf-8')

print("✓ Fungsi decrypt_text berhasil dibuat!")


✓ Fungsi decrypt_text berhasil dibuat!


In [34]:
def encrypt_file(input_file: str, output_file: str, password: str):
    if os.path.getsize(input_file) > 1024 * 1024:
        raise ValueError("Ukuran file melebihi 1 MB")

    with open(input_file, 'r', encoding='utf-8') as f:
        plaintext = f.read()

    encrypted_data = encrypt_text(plaintext, password)

    with open(output_file, 'w') as f:
        json.dump(encrypted_data, f, indent=2)

    print(f"✓ File berhasil dienkripsi: {output_file}")
    return encrypted_data

print("✓ Fungsi encrypt_file berhasil dibuat!")


✓ Fungsi encrypt_file berhasil dibuat!


In [35]:
def decrypt_file(input_file: str, output_file: str, password: str):
    with open(input_file, 'r') as f:
        encrypted_data = json.load(f)

    plaintext = decrypt_text(encrypted_data, password)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(plaintext)

    print(f"✓ File berhasil didekripsi: {output_file}")
    return plaintext

print("✓ Fungsi decrypt_file berhasil dibuat!")


✓ Fungsi decrypt_file berhasil dibuat!


In [36]:
def main():
    print("=" * 70)
    print("   APLIKASI KRIPTOGRAFI AES-256 CBC - Data Science Telkom")
    print("=" * 70)

    while True:
        print("\n" + "=" * 60)
        print("MENU: 1.Enkripsi Teks | 2.Dekripsi Teks")
        print("      3.Enkripsi File | 4.Dekripsi File | 0.Keluar")
        print("=" * 60)

        choice = input("\nPilih (0-4): ").strip()

        try:
            if choice == '1':
                plaintext = input("Teks yang akan dienkripsi: ")
                password = input("Password: ")

                if not plaintext or not password:
                    print("❌ Input tidak boleh kosong!")
                    continue

                encrypted = encrypt_text(plaintext, password)

                output_file = f"encrypted_text_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
                with open(output_file, 'w') as f:
                    json.dump(encrypted, f, indent=2)

                print(f"\n✓ BERHASIL DIENKRIPSI!")
                print(f"File tersimpan: {output_file}")
                print(f"\nIsi file JSON:")
                print(f"  Ciphertext: {encrypted['ciphertext'][:60]}...")
                print(f"  IV: {encrypted['iv']}")
                print(f"  Salt: {encrypted['salt']}")

            elif choice == '2':
                print("\n📤 Upload file JSON hasil enkripsi...")
                json_file = upload_file()

                if not json_file:
                    print("❌ Upload dibatalkan!")
                    continue

                with open(json_file, 'r') as f:
                    encrypted_data = json.load(f)

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                plaintext = decrypt_text(encrypted_data, password)

                output_file = f"decrypted_text_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(plaintext)

                print(f"\n✓ BERHASIL DIDEKRIPSI!")
                print(f"File tersimpan: {output_file}")
                print(f"\nHasil dekripsi:")
                print(f"  {plaintext}")

            elif choice == '3':
                print("\n📤 Upload file yang akan dienkripsi...")
                input_file = upload_file()

                if not input_file:
                    print("❌ Upload dibatalkan!")
                    continue

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                with open(input_file, 'rb') as f:
                    original_content = f.read()

                plaintext = original_content.decode('utf-8', errors='ignore')
                encrypted = encrypt_text(plaintext, password)

                output_file = f"{input_file}_encrypted.json"
                metadata = {
                    'original_filename': input_file,
                    'ciphertext': encrypted['ciphertext'],
                    'iv': encrypted['iv'],
                    'salt': encrypted['salt']
                }

                with open(output_file, 'w') as f:
                    json.dump(metadata, f, indent=2)

                print(f"\n✓ FILE BERHASIL DIENKRIPSI!")
                print(f"File input: {input_file}")
                print(f"File output: {output_file}")

            elif choice == '4':
                print("\n📤 Upload file JSON terenkripsi...")
                json_file = upload_file()

                if not json_file:
                    print("❌ Upload dibatalkan!")
                    continue

                with open(json_file, 'r') as f:
                    metadata = json.load(f)

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                encrypted_data = {
                    'ciphertext': metadata['ciphertext'],
                    'iv': metadata['iv'],
                    'salt': metadata['salt']
                }

                plaintext = decrypt_text(encrypted_data, password)

                original_filename = metadata['original_filename']
                output_file = f"decrypted_{original_filename}"

                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(plaintext)

                print(f"\n✓ FILE BERHASIL DIDEKRIPSI!")
                print(f"File output: {output_file}")
                print(f"File format asli: {original_filename}")

            elif choice == '0':
                break
            else:
                print("❌ Pilihan tidak valid!")

        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            print("💡 Tips: Pastikan password benar dan file format valid")


In [37]:
main()

   APLIKASI KRIPTOGRAFI AES-256 CBC - Data Science Telkom

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 1
Teks yang akan dienkripsi: Halo Semuanya Saya Fauzan
Password: 1234

✓ BERHASIL DIENKRIPSI!
File tersimpan: encrypted_text_20251103_063527.json

Isi file JSON:
  Ciphertext: nKvs3gO7d5O03MPRcJ+BfDqu6U3ASwH84aSQnOEQvVQ=...
  IV: DA4V+HEd0qoipUDbCzGf2A==
  Salt: Ns1hVZOMLtLOI7/3exrMEQ==

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 2

📤 Upload file JSON hasil enkripsi...
📤 Silakan pilih file untuk diupload...


Saving encrypted_text_20251103_063527.json to encrypted_text_20251103_063527 (1).json
✓ File berhasil diupload: encrypted_text_20251103_063527 (1).json
Password: 1234

✓ BERHASIL DIDEKRIPSI!
File tersimpan: decrypted_text_20251103_063549.txt

Hasil dekripsi:
  Halo Semuanya Saya Fauzan

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 3

📤 Upload file yang akan dienkripsi...
📤 Silakan pilih file untuk diupload...


Saving fau_4.csv to fau_4.csv
✓ File berhasil diupload: fau_4.csv
Password: 1234

✓ FILE BERHASIL DIENKRIPSI!
File input: fau_4.csv
File output: fau_4.csv_encrypted.json

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 4

📤 Upload file JSON terenkripsi...
📤 Silakan pilih file untuk diupload...


Saving fau_4.csv_encrypted.json to fau_4.csv_encrypted (1).json
✓ File berhasil diupload: fau_4.csv_encrypted (1).json
Password: 1234

✓ FILE BERHASIL DIDEKRIPSI!
File output: decrypted_fau_4.csv
File format asli: fau_4.csv

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 0
